In [1]:
import tensorflow as tf
import glob, os
import tqdm
import os
import numpy as np
from PIL import Image

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import tensorflow as tf

from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization, LeakyReLU

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import cv2
from google.colab.patches import cv2_imshow

import imageio
import skimage
import skimage.io
import skimage.transform

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Capstone Project C23-PC648/Notebook

Mounted at /content/drive
/content/drive/MyDrive/Capstone Project C23-PC648/Notebook


# 1. Read Data

In [8]:
bee_data = pd.read_csv('bee_data.csv')
bee_data.head()

,file,date,time,location,zip code,subspecies,health,pollen_carrying,caste
0,041_066.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
1,041_072.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
2,041_073.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
3,041_067.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
4,041_059.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker


# 2. Split Train and Test Data

In [9]:
RANDOM_SEED = 123
TEST_SIZE = 0.2
TRAIN_SIZE = 0.8

In [10]:
train_data, test_data = train_test_split(
    bee_data, 
    test_size = TEST_SIZE, 
    stratify = bee_data['health'],
    random_state = RANDOM_SEED
    )

In [11]:
train_data.head()

,file,date,time,location,zip code,subspecies,health,pollen_carrying,caste
1378,005_599.png,7/7/18,12:56,"Saratoga, CA, USA",95070,Italian honey bee,healthy,False,worker
4185,038_202.png,8/18/18,12:30,"Athens, Georgia, USA",30607,1 Mixed local stock 2,"Varroa, Small Hive Beetles",False,worker
1922,040_457.png,8/21/18,15:56,"Athens, GA, USA",30607,Italian honey bee,"few varrao, hive beetles",False,worker
3865,032_931.png,8/21/18,9:00,"Des Moines, IA, USA",50315,Russian honey bee,healthy,False,worker
253,003_145.png,7/2/18,13:57,"Saratoga, CA, USA",95070,Italian honey bee,healthy,False,worker


In [12]:
test_data.head()

,file,date,time,location,zip code,subspecies,health,pollen_carrying,caste
3437,015_1235.png,8/6/18,13:19,"Saratoga, CA, USA",95070,Italian honey bee,healthy,False,worker
1564,040_066.png,8/21/18,15:56,"Athens, GA, USA",30607,Italian honey bee,"few varrao, hive beetles",False,worker
3793,032_666.png,8/21/18,9:00,"Des Moines, IA, USA",50315,Russian honey bee,healthy,False,worker
2047,016_139.png,8/20/18,11:55,"San Jose, CA, USA",95124,-1,healthy,False,worker
3395,015_1222.png,8/6/18,13:19,"Saratoga, CA, USA",95070,Italian honey bee,healthy,False,worker


In [13]:
train_data['health'].value_counts()

healthy                       2707
few varrao, hive beetles       463
Varroa, Small Hive Beetles     377
ant problems                   366
hive being robbed              201
missing queen                   23
Name: health, dtype: int64

In [14]:
test_data['health'].value_counts()

healthy                       677
few varrao, hive beetles      116
Varroa, Small Hive Beetles     95
ant problems                   91
hive being robbed              50
missing queen                   6
Name: health, dtype: int64

# 3. Convert Data to NPY

In [16]:
from PIL import Image

In [21]:
from keras.utils import load_img
from keras.utils import img_to_array

In [15]:
IMAGE_WIDTH = 96
IMAGE_HEIGHT = 96

In [16]:
def read_image(file_name):
  image = skimage.io.imread('bee_imgs/' + file_name)
  image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), mode='reflect')
  return image[:,:,:3]

In [17]:
X_train = np.stack(train_data['file'].apply(read_image))
y_train = pd.get_dummies(train_data['health'], drop_first=False)

In [18]:
X_test = np.stack(test_data['file'].apply(read_image))
y_test = pd.get_dummies(test_data['health'], drop_first=False)

In [19]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4137, 96, 96, 3) (4137, 6) (1035, 96, 96, 3) (1035, 6)


In [20]:
y_train.to_pickle('y_train.pkl')
y_test.to_pickle('y_test.pkl')

In [21]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)
with open('y_train.npy', 'wb') as f:
    np.save(f, y_train)

In [22]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)
with open('y_test.npy', 'wb') as f:
    np.save(f, y_test)

In [32]:
# X_train = np.load('SAVE_NPY_1/X_train.npy')
# y_train = pd.read_pickle('SAVE_NPY_1/y_train.pkl')
# X_test  = np.load('SAVE_NPY_1/X_test.npy')
# y_test  = pd.read_pickle('SAVE_NPY_1/y_test.pkl')

In [33]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4137, 100, 100, 3) (4137, 6) (1035, 100, 100, 3) (1035, 6)


In [23]:
generator = ImageDataGenerator(
    # scale=1./255,
    # featurewise_center=False, 
    # samplewise_center=False, 
    # featurewise_std_normalization=False, 
    # samplewise_std_normalization=False, 
    # zca_whitening=False, 
    rotation_range=180, 
    zoom_range = 0.1,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True, 
    vertical_flip=True)
generator.fit(X_train)

# Custom 1

In [26]:
def create_model():
  model = Sequential([
      tf.keras.layers.Conv2D(16, kernel_size=3, 
                             input_shape=(96, 96, 3),
                             activation='relu', padding='same'),
      tf.keras.layers.MaxPool2D(2),

      tf.keras.layers.Conv2D(16, kernel_size=3,
                             activation='relu', padding='same'),
      tf.keras.layers.MaxPool2D(2),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(6, activation='softmax')
  ])

  return model

In [27]:
tf.keras.backend.clear_session()

model = create_model()

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
BATCH_SIZE = 32

In [29]:
history  = model.fit(
    generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=20,
    validation_data=[X_test, y_test],
    steps_per_epoch=len(X_train)/BATCH_SIZE
    )

Epoch 1/20
129/129 [==============================] - 49s 370ms/step - loss: 0.9185 - accuracy: 0.6853 - val_loss: 0.6480 - val_accuracy: 0.7758
Epoch 2/20
129/129 [==============================] - 49s 379ms/step - loss: 0.5484 - accuracy: 0.7810 - val_loss: 0.4627 - val_accuracy: 0.7990
Epoch 3/20
129/129 [==============================] - 45s 345ms/step - loss: 0.4600 - accuracy: 0.8090 - val_loss: 0.3900 - val_accuracy: 0.8261
Epoch 4/20
129/129 [==============================] - 48s 370ms/step - loss: 0.3954 - accuracy: 0.8322 - val_loss: 0.3342 - val_accuracy: 0.8541
Epoch 5/20
129/129 [==============================] - 45s 344ms/step - loss: 0.3302 - accuracy: 0.8501 - val_loss: 0.3102 - val_accuracy: 0.8493
Epoch 6/20
129/129 [==============================] - 45s 343ms/step - loss: 0.3341 - accuracy: 0.8521 - val_loss: 0.3032 - val_accuracy: 0.8628
Epoch 7/20
129/129 [==============================] - 45s 350ms/step - loss: 0.2953 - accuracy: 0.8661 - val_loss: 0.2823 - val_ac

In [30]:
model.save('model_custom1.h5')

# Custom 2

In [38]:
def create_model_2():
  model = Sequential([
    tf.keras.layer.Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(96, 96, 3)),

    # convolutional layer
    tf.keras.layer.Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
    tf.keras.layer.MaxPool2D(pool_size=(2,2)),
    tf.keras.layer.Dropout(0.25),

    tf.keras.layer.Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
    tf.keras.layer.MaxPool2D(pool_size=(2,2)),
    tf.keras.layer.Dropout(0.25),

    # flatten output of conv
    tf.keras.layer.Flatten(),

    # hidden layer
    tf.keras.layer.Dense(500, activation='relu'),
    tf.keras.layer.Dropout(0.4),
    tf.keras.layer.Dense(250, activation='relu'),
    tf.keras.layer.Dropout(0.3),

    # output layer
    tf.keras.layer.Dense(6, activation='softmax'),
  ])

  return model

In [41]:
tf.keras.backend.clear_session()

model_2 = create_model()

model_2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [42]:
train_model  = model_2.fit(
    generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=25,
    validation_data=[X_test, y_test],
    steps_per_epoch=len(X_train)/BATCH_SIZE
    )

Epoch 1/25
129/129 [==============================] - 45s 338ms/step - loss: 0.8587 - accuracy: 0.6998 - val_loss: 0.5749 - val_accuracy: 0.7720
Epoch 2/25
129/129 [==============================] - 44s 342ms/step - loss: 0.4471 - accuracy: 0.8226 - val_loss: 0.3968 - val_accuracy: 0.8415
Epoch 3/25
129/129 [==============================] - 45s 346ms/step - loss: 0.3467 - accuracy: 0.8557 - val_loss: 0.4157 - val_accuracy: 0.8077
Epoch 4/25
129/129 [==============================] - 44s 338ms/step - loss: 0.3114 - accuracy: 0.8639 - val_loss: 0.2829 - val_accuracy: 0.8696
Epoch 5/25
129/129 [==============================] - 45s 349ms/step - loss: 0.2883 - accuracy: 0.8714 - val_loss: 0.2366 - val_accuracy: 0.8899
Epoch 6/25
129/129 [==============================] - 45s 346ms/step - loss: 0.2569 - accuracy: 0.8837 - val_loss: 0.2299 - val_accuracy: 0.8889
Epoch 7/25
129/129 [==============================] - 44s 337ms/step - loss: 0.2582 - accuracy: 0.8849 - val_loss: 0.2280 - val_ac

In [49]:
model_2.save('model_custom2.h5')

# MobileNetV2

In [44]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(96,96,3),
    include_top=False,
    weights='imagenet'
    )

base_model.trainable = True
for layer in base_model.layers[:120]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(96,96,3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dropout(.2)(x)
prediction_layer = tf.keras.layers.Dense(6, activation='softmax')

outputs = prediction_layer(x) 
model = tf.keras.Model(inputs, outputs)

9406464/9406464 [==============================] - 0s 0us/step


In [45]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 96, 96, 3)        0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 96, 96, 3)        0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                      

In [46]:
base_learning_rate = 0.01
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
BATCH_SIZE = 32

In [48]:
train_model  = model.fit(
    generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=10,
    validation_data=[X_test, y_test],
    steps_per_epoch=len(X_train)/BATCH_SIZE
    )

Epoch 1/10
129/129 [==============================] - 88s 617ms/step - loss: 1.7364 - accuracy: 0.5255 - val_loss: 1.3169 - val_accuracy: 0.6541
Epoch 2/10
129/129 [==============================] - 80s 617ms/step - loss: 1.4427 - accuracy: 0.5741 - val_loss: 1.2226 - val_accuracy: 0.6541
Epoch 3/10
129/129 [==============================] - 76s 584ms/step - loss: 1.3524 - accuracy: 0.6048 - val_loss: 1.1430 - val_accuracy: 0.6541
Epoch 4/10
129/129 [==============================] - 75s 577ms/step - loss: 1.2789 - accuracy: 0.6268 - val_loss: 1.1679 - val_accuracy: 0.6541
Epoch 5/10
129/129 [==============================] - 77s 596ms/step - loss: 1.2586 - accuracy: 0.6454 - val_loss: 1.1513 - val_accuracy: 0.6541
Epoch 6/10
129/129 [==============================] - 75s 582ms/step - loss: 1.2197 - accuracy: 0.6517 - val_loss: 1.1501 - val_accuracy: 0.6541
Epoch 7/10
129/129 [==============================] - 75s 577ms/step - loss: 1.2135 - accuracy: 0.6539 - val_loss: 1.1558 - val_ac

In [50]:
model.save('model_mbnv2.h5')

# VGG-16

In [54]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input
base_model = tf.keras.applications.VGG16(
    input_shape=(96,96,3),
    include_top=False,
    weights='imagenet'
    )

base_model.trainable = False

In [55]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 48, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 48, 48, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 24, 24, 128)       0     

In [73]:
tf.keras.backend.clear_session()

inputs = tf.keras.Input(shape=(96,96,3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(50, activation='relu')(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)
prediction_layer = tf.keras.layers.Dense(6, activation='softmax')

outputs = prediction_layer(x) 
model_vgg16 = tf.keras.Model(inputs, outputs)

In [70]:
model = Sequential([
    # tf.keras.applications.vgg16.preprocess_input,

    base_model,
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(6, activation='softmax')
])

In [74]:
model_vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 tf.__operators__.getitem (S  (None, 96, 96, 3)        0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 96, 96, 3)        0         
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 50)                230450    
                                                             

In [75]:
model_vgg16.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [76]:
BATCH_SIZE = 32

In [ ]:
train_model  = model_vgg16.fit(
    generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=10,
    validation_data=[X_test, y_test],
    steps_per_epoch=len(X_train)/BATCH_SIZE
    )

Epoch 1/10
129/129 [==============================] - 637s 5s/step - loss: 1.1817 - accuracy: 0.6543 - val_loss: 1.1392 - val_accuracy: 0.6541
Epoch 2/10
129/129 [==============================] - 633s 5s/step - loss: 1.1390 - accuracy: 0.6543 - val_loss: 1.1384 - val_accuracy: 0.6541
Epoch 3/10
129/129 [==============================] - 634s 5s/step - loss: 1.1327 - accuracy: 0.6543 - val_loss: 1.1305 - val_accuracy: 0.6541
Epoch 4/10
129/129 [==============================] - 612s 5s/step - loss: 1.1342 - accuracy: 0.6543 - val_loss: 1.1242 - val_accuracy: 0.6541
Epoch 5/10
129/129 [==============================] - 636s 5s/step - loss: 1.1360 - accuracy: 0.6543 - val_loss: 1.1808 - val_accuracy: 0.6541
Epoch 6/10
129/129 [==============================] - 638s 5s/step - loss: 1.1332 - accuracy: 0.6543 - val_loss: 1.1232 - val_accuracy: 0.6541
Epoch 7/10
129/129 [==============================] - 634s 5s/step - loss: 1.1288 - accuracy: 0.6543 - val_loss: 1.1274 - val_accuracy: 0.6541